# 10. data preprocessing to save time in application-level evaluations

In [1]:
import uavgeo as ug
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import os
import pandas as pd

c:\users\judoj\documents\programming\uavgeo\uavgeo\compute\products.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## 1. BTG2022 preprocessing

In [3]:
# LOAD DATA bt2022

ndvi_p2phd = rxr.open_rasterio("../data/orthos/reconstructed/ndvi_p2p_hd_2022.tif",mode = "w")
ndvi_p2p = rxr.open_rasterio("../data/orthos/reconstructed/ndvi_p2p_v2_2022.tif",mode = "w")
ndvi_test = rxr.open_rasterio("../data/orthos/ndvi_2022_test.tif",mode = "w")
rgb = rxr.open_rasterio("../data/orthos/rgb_2022.tif",mode = "w")

chm = rxr.open_rasterio("../data/orthos/rgb_2022_chm.tif",mode = "w")

dem = rxr.open_rasterio("../data/orthos/rgb_2022_dem.tif",mode = "w")
dsm = rxr.open_rasterio("../data/orthos/rgb_2022_dsm.tif",mode = "w")
chm = rxr.open_rasterio("../data/orthos/rgb_2022_chm.tif",mode = "w")

In [4]:
# align reproject etc.
#pixel-level alignment of the dem,dsm,chm rasters:

chm = chm.rio.reproject_match(ndvi_test)
dem = dem.rio.reproject_match(ndvi_test)
dsm = dsm.rio.reproject_match(ndvi_test)
rgb = rgb.rio.reproject_match(ndvi_test)

ndvi_p2p = ndvi_p2p.rio.reproject_match(ndvi_test)
ndvi_p2phd = ndvi_p2phd.rio.reproject_match(ndvi_test)

In [5]:

roi = gpd.read_file("../data/bbr/btg2022/roi/RoI.shp")
#clip all to the same extent
ndvi_test = ndvi_test.rio.clip(roi.geometry)
ndvi_p2phd = ndvi_p2phd.rio.clip(roi.geometry)
ndvi_p2p = ndvi_p2p.rio.clip(roi.geometry)
chm = chm.rio.clip(roi.geometry)
rgb = rgb.rio.clip(roi.geometry)

dem = dem.rio.clip(roi.geometry)
dsm = dsm.rio.clip(roi.geometry)

C:\Users\judoj\mambaforge\envs\ndvi\Lib\site-packages\rioxarray\raster_writer.py:130: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(
C:\Users\judoj\mambaforge\envs\ndvi\Lib\site-packages\rioxarray\raster_writer.py:130: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


In [6]:
#cset nodata value
#cset nodata value
ndvi_test = ndvi_test.rio.write_nodata(ndvi_test.rio.nodata, encoded=True)
ndvi_p2phd = ndvi_p2phd.rio.write_nodata(ndvi_p2phd.rio.nodata, encoded=True)
ndvi_p2p = ndvi_p2p.rio.write_nodata(ndvi_p2p.rio.nodata, encoded=True)
dem = dem.rio.write_nodata(dem.rio.nodata, encoded=True)
chm = chm.rio.write_nodata(chm.rio.nodata, encoded=True)
dsm = dsm.rio.write_nodata(dsm.rio.nodata, encoded=True)
rgb = rgb.rio.write_nodata(rgb.rio.nodata, encoded=True)

In [7]:
# add rgbvi into the mix
rgbvi = xr.full_like(ndvi_test, fill_value = np.nan)
vals = ug.compute.calc_rgbvi(rgb, red_id = 1, green_id = 2, blue_id= 3, rescale = False).values
rgbvi.values = vals.reshape(rgbvi.values.shape)

In [8]:
ndvi_test.rio.to_raster("../data/orthos/preprocessed/ndvi_test.tif")
ndvi_p2p.rio.to_raster("../data/orthos/preprocessed/ndvi_p2p.tif")
ndvi_p2phd.rio.to_raster("../data/orthos/preprocessed/ndvi_p2phd.tif")
chm.rio.to_raster("../data/orthos/preprocessed/chm.tif")
rgbvi.rio.to_raster("../data/orthos/preprocessed/rgbvi.tif")
dsm.rio.to_raster("../data/orthos/preprocessed/dsm.tif")
dem.rio.to_raster("../data/orthos/preprocessed/dem.tif")
rgb.rio.to_raster("../data/orthos/preprocessed/rgb.tif")

## 2. CAN2023 preprocessing

In [3]:
# load data can2023

ndvi_p2phd = rxr.open_rasterio("../data/orthos/reconstructed/canyelles_ndvi_p2p_hd.tif",mode = "r", chunks=True).sel(band=[1])
ndvi_p2p = rxr.open_rasterio("../data/orthos/reconstructed/canyelles_ndvi_p2p.tif",mode = "r", chunks=True).sel(band=[1])
ndvi_test = rxr.open_rasterio("../data/orthos/reconstructed/canyelles_ndvi_real.tif",mode = "r", chunks=True).sel(band=[1])

dem = rxr.open_rasterio("../data/canyelles/orthos/can_2023_dem.tif",mode = "r", chunks=True).astype(np.float32)
dem = dem.rio.write_nodata(dem.rio.nodata, encoded=True)
dsm = rxr.open_rasterio("../data/canyelles/orthos/can_2023_dsm.tif",mode = "r", chunks=True).astype(np.float32)
dsm = dsm.rio.write_nodata(dsm.rio.nodata, encoded=True)
chm = rxr.open_rasterio("../data/canyelles/orthos/can_2023_chm.tif",mode = "r", chunks=True).astype(np.float32)
chm = chm.rio.write_nodata(chm.rio.nodata, encoded=True)

rgb = rxr.open_rasterio("../data/canyelles/orthos/rgb_230609.tif",mode = "r", chunks=True).astype(np.uint8)
rgb = rgb.rio.write_nodata(rgb.rio.nodata, encoded=True)
#and the mask
mask = gpd.read_file("../data/canyelles/shapes/vineyard_shape.geojson")

In [4]:
ndvi_test = ndvi_test.rio.reproject("EPSG:32631")
ndvi_p2p = ndvi_p2p.rio.reproject("EPSG:32631")
ndvi_p2phd = ndvi_p2phd.rio.reproject("EPSG:32631")
rgb = rgb.rio.reproject("EPSG:32631")

In [5]:
#cset nodata value
ndvi_test = ndvi_test.rio.write_nodata(ndvi_test.rio.nodata, encoded=True)
ndvi_p2phd = ndvi_p2phd.rio.write_nodata(ndvi_p2phd.rio.nodata, encoded=True)
ndvi_p2p = ndvi_p2p.rio.write_nodata(ndvi_p2p.rio.nodata, encoded=True)
chm = chm.rio.write_nodata(chm.rio.nodata, encoded=True)
rgb = rgb.rio.write_nodata(rgb.rio.nodata, encoded=True)

C:\Users\judoj\mambaforge\envs\ndvi\Lib\site-packages\rioxarray\raster_writer.py:130: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


In [6]:
mask = mask.to_crs("EPSG:32631")
mask = mask.geometry

In [7]:
# add rgbvi into the mix
#rgbvi = xr.full_like(ndvi_test, fill_value = np.nan)
rgbvi = ug.compute.calc_rgbvi(rgb, red_id = 1, green_id = 2, blue_id= 3, rescale = False)
#rgbvi.values = vals.reshape(rgbvi.values.shape)

In [8]:
rgbvi = rgbvi.astype(np.float32)

In [9]:
# align to same pixel-grid
#pixel-level alignment 
chm = chm.rio.reproject_match(ndvi_test)
dem = dem.rio.reproject_match(ndvi_test)
dsm = dsm.rio.reproject_match(ndvi_test)
#RGB
rgbvi = rgbvi.rio.reproject_match(ndvi_test,nodata= 0)
ndvi_p2p = ndvi_p2p.rio.reproject_match(ndvi_test)
ndvi_p2phd = ndvi_p2phd.rio.reproject_match(ndvi_test)

In [10]:
vals = rgbvi.values
rgbvi = xr.full_like(ndvi_test, fill_value = np.nan)
rgbvi.values = vals.reshape(rgbvi.values.shape)

In [11]:
#cset nodata value again
ndvi_test = ndvi_test.rio.write_nodata(ndvi_test.rio.nodata, encoded=True)
ndvi_p2phd = ndvi_p2phd.rio.write_nodata(ndvi_p2phd.rio.nodata, encoded=True)
ndvi_p2p = ndvi_p2p.rio.write_nodata(ndvi_p2p.rio.nodata, encoded=True)
dem = dem.rio.write_nodata(dem.rio.nodata, encoded=True)
chm = chm.rio.write_nodata(chm.rio.nodata, encoded=True)
dsm = dsm.rio.write_nodata(dsm.rio.nodata, encoded=True)
rgbvi = rgbvi.rio.write_nodata(rgbvi.rio.nodata, encoded=True)

C:\Users\judoj\mambaforge\envs\ndvi\Lib\site-packages\rioxarray\raster_writer.py:130: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


In [13]:
#mask that shittt
ndvi_test = ndvi_test.rio.clip(mask)
ndvi_p2p = ndvi_p2p.rio.clip(mask)
ndvi_p2phd = ndvi_p2phd.rio.clip(mask)
#mask that shittt
chm= chm.rio.clip(mask)
dem = dem.rio.clip(mask)
dsm = dsm.rio.clip(mask)

#rgb
#rgb = rgb.rio.clip(mask)
rgbvi = rgbvi.rio.clip(mask)

In [14]:
#cset nodata value again
ndvi_test = ndvi_test.rio.write_nodata(ndvi_test.rio.nodata, encoded=True)
ndvi_p2phd = ndvi_p2phd.rio.write_nodata(ndvi_p2phd.rio.nodata, encoded=True)
ndvi_p2p = ndvi_p2p.rio.write_nodata(ndvi_p2p.rio.nodata, encoded=True)
dem = dem.rio.write_nodata(dem.rio.nodata, encoded=True)
chm = chm.rio.write_nodata(chm.rio.nodata, encoded=True)
dsm = dsm.rio.write_nodata(dsm.rio.nodata, encoded=True)
rgbvi= rgbvi.rio.write_nodata(rgbvi.rio.nodata, encoded = True)

In [15]:
ndvi_test.rio.to_raster("../data/canyelles/orthos/preprocessed/ndvi_test.tif")
ndvi_p2p.rio.to_raster("../data/canyelles/orthos/preprocessed/ndvi_p2p.tif")
ndvi_p2phd.rio.to_raster("../data/canyelles/orthos/preprocessed/ndvi_p2phd.tif")
chm.rio.to_raster("../data/canyelles/orthos/preprocessed/chm.tif")
dem.rio.to_raster("../data/canyelles/orthos/preprocessed/dem.tif")
dsm.rio.to_raster("../data/canyelles/orthos/preprocessed/dsm.tif")
rgbvi.rio.to_raster("../data/canyelles/orthos/preprocessed/rgbvi.tif")

In [3]:
#single cell rgb preprocessing
mask = gpd.read_file("../data/canyelles/shapes/vineyard_shape.geojson")
mask = mask.to_crs("EPSG:32631")
mask = mask.geometry

rgb = rxr.open_rasterio("../data/canyelles/orthos/rgb_230609.tif",mode = "r", chunks=True).astype(np.float32)
rgb = rgb.rio.write_nodata(rgb.rio.nodata, encoded=True)
rgb = rgb.rio.reproject("EPSG:32631")

ndvi_test = rxr.open_rasterio("../data/canyelles/orthos/preprocessed/ndvi_test.tif")
rgb = rgb.rio.reproject_match(ndvi_test)
rgb = rgb.rio.clip(mask)
rgb = rgb.rio.write_nodata(rgb.rio.nodata, encoded=True)
rgb = rgb.astype(np.uint8)
rgb.rio.to_raster("../data/canyelles/orthos/preprocessed/rgb.tif")

C:\Users\judoj\mambaforge\envs\ndvi\Lib\site-packages\rioxarray\raster_writer.py:130: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


## 2. btg2021-test preprocessing

In [3]:
# load data can2023

ndvi_p2phd = rxr.open_rasterio("../data/orthos/reconstructed/ndvi_hd_p2p_2021.tif",mode = "r", chunks=True).sel(band=[1])
ndvi_p2p = rxr.open_rasterio("../data/orthos/reconstructed/ndvi_p2p2021.tif",mode = "r", chunks=True).sel(band=[1])
ndvi_test = rxr.open_rasterio("../data/orthos/reconstructed/ndvi_test2021.tif",mode = "r", chunks=True).sel(band=[1])

dem = rxr.open_rasterio("../data/orthos/btg_2021_dem.tif",mode = "r", chunks=True)
dem = dem.rio.write_nodata(dem.rio.nodata, encoded=True)
dsm = rxr.open_rasterio("../data/orthos/dsm_2021.tif",mode = "r", chunks=True)

chm =rxr.open_rasterio("../data/orthos/btg_2021_chm.tif",mode = "r", chunks=True)
chm = chm.rio.write_nodata(chm.rio.nodata, encoded=True)

#and the mask
roi = gpd.read_file("../data/bbr/btg2021_test/roi.geojson")

In [4]:
ortho  = rxr.open_rasterio("../data/orthos/ms_2021.tif",mode = "r", chunks=True).sel(band=[1,2,3])
r = 7500
g = 7500
b = 6000
r_new = ug.compute.scale_band_to_min_max(ortho.sel(band=3), min = 0, max = r)
g_new =  ug.compute.scale_band_to_min_max(ortho.sel(band=2), min = 0, max = g)
b_new =  ug.compute.scale_band_to_min_max(ortho.sel(band=1), min = 0, max = b)
rgb = xr.combine_nested([r_new, g_new, b_new],concat_dim = "band")

In [5]:
# add rgbvi into the mix
rgbvi = xr.full_like(rgb.sel(band=1), fill_value = np.nan)
vals = ug.compute.calc_rgbvi(rgb, red_id = 1, green_id = 2, blue_id= 3, rescale = False).values
rgbvi.values = vals.reshape(rgbvi.values.shape)

C:\Users\judoj\mambaforge\envs\ndvi\Lib\site-packages\dask\core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
C:\Users\judoj\mambaforge\envs\ndvi\Lib\site-packages\numpy\core\numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')


In [6]:


#clip all to the same extent
ndvi_test = ndvi_test.rio.clip(roi.geometry)
ndvi_p2phd = ndvi_p2phd.rio.clip(roi.geometry)
ndvi_p2p = ndvi_p2p.rio.clip(roi.geometry)
chm = chm.rio.clip(roi.geometry)
rgb = rgb.rio.clip(roi.geometry)
rgbvi = rgbvi.rio.clip(roi.geometry)
dem = dem.rio.clip(roi.geometry)
dsm = dsm.rio.clip(roi.geometry)

In [7]:
# align reproject etc.
#pixel-level alignment of the dem,dsm,chm rasters:

chm = chm.rio.reproject_match(ndvi_test)
dem = dem.rio.reproject_match(ndvi_test)
dsm = dsm.rio.reproject_match(ndvi_test)
rgb = rgb.rio.reproject_match(ndvi_test)
rgbvi = rgbvi.rio.reproject_match(ndvi_test)
ndvi_test = ndvi_test.rio.reproject_match(ndvi_test)
ndvi_p2p = ndvi_p2p.rio.reproject_match(ndvi_test)
ndvi_p2phd = ndvi_p2phd.rio.reproject_match(ndvi_test)

C:\Users\judoj\mambaforge\envs\ndvi\Lib\site-packages\dask\array\chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)


In [8]:
#cset nodata value
#cset nodata value
ndvi_test = ndvi_test.rio.write_nodata(ndvi_test.rio.nodata, encoded=True)
ndvi_p2phd = ndvi_p2phd.rio.write_nodata(ndvi_p2phd.rio.nodata, encoded=True)
ndvi_p2p = ndvi_p2p.rio.write_nodata(ndvi_p2p.rio.nodata, encoded=True)
dem = dem.rio.write_nodata(dem.rio.nodata, encoded=True)
chm = chm.rio.write_nodata(chm.rio.nodata, encoded=True)
dsm = dsm.rio.write_nodata(dsm.rio.nodata, encoded=True)
rgb = rgb.rio.write_nodata(rgb.rio.nodata, encoded=True)
rgbvi = rgbvi.rio.write_nodata(rgbvi.rio.nodata, encoded=True)

C:\Users\judoj\mambaforge\envs\ndvi\Lib\site-packages\rioxarray\raster_writer.py:130: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


In [11]:
ndvi_test.rio.to_raster("../data/orthos/preprocessed/btg_2021_test/ndvi_test.tif")
ndvi_p2p.rio.to_raster("../data/orthos/preprocessed/btg_2021_test/ndvi_p2p.tif")
ndvi_p2phd.rio.to_raster("../data/orthos/preprocessed/btg_2021_test/ndvi_p2phd.tif")
chm.rio.to_raster("../data/orthos/preprocessed/btg_2021_test/chm.tif")
rgbvi.rio.to_raster("../data/orthos/preprocessed/btg_2021_test/rgbvi.tif")
dsm.rio.to_raster("../data/orthos/preprocessed/btg_2021_test/dsm.tif")
dem.rio.to_raster("../data/orthos/preprocessed/btg_2021_test/dem.tif")
rgb.rio.to_raster("../data/orthos/preprocessed/btg_2021_test/rgb.tif")